<a href="https://colab.research.google.com/github/mekhi-woods/HiloCATsSN1991bg/blob/master/amadeus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# """ START UP """
# import os
# import shutil
# if os.path.exists('/content/HiloCATsSN1991bg') == True:
#     shutil.rmtree('/content/HiloCATsSN1991bg')
#     !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git
# else:
#     !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git

# !pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple snpy
# !pip install requests
# !pip install sncosmo
# !pip install iminuit

In [ ]:
""" IMPORTS """
import os
import time
import glob
import json
import snpy
import shutil
import sncosmo
import requests
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from zipfile import ZipFile
from requests.auth import HTTPBasicAuth
from HiloCATsSN1991bg.scripts import tns_redshifts
from astropy.table import QTable, Table, Column
from astropy import units as u

In [ ]:
""" GLOBALS """
# TNS CREDINTIALS
tns_bot_id = '73181'
tns_bot_name = 'YSE_Bot1'
tns_bot_api_key = '0d771345fa6b876a5bb99cd5042ab8b5ae91fc67'

# PATHS
ROOT_PATH = '/content/HiloCATsSN1991bg/'


PROBLEM_CHILDREN = ['1032212120425304400']  # Running list of problematic SNe
                                            # '1032212120425304400' - David believes it might be a shock breakout but ATLAS reports it as SN1a
                                            #


In [ ]:
""" GENERAL """
def recover_dir():
    directories = [ROOT_PATH]
    for dir in directories:
        if os.path.exists(dir) == False:
            os.mkdir(dir)
    return

def TNS_details(ra, dec):
    # Code from David
    headers = tns_redshifts.build_tns_header(tns_bot_id, tns_bot_name)
    tns_api_url = f"https://www.wis-tns.org/api/get"

    # get the API URLs
    search_tns_url = tns_redshifts.build_tns_url(tns_api_url, mode="search")
    get_tns_url = tns_redshifts.build_tns_url(tns_api_url, mode="get")

    search_data = tns_redshifts.build_tns_search_query_data(tns_bot_api_key, ra, dec)
    transients = tns_redshifts.rate_limit_query_tns(search_data, headers, search_tns_url)

    get_data = tns_redshifts.build_tns_get_query_data(tns_bot_api_key, transients[0])
    transient_detail = tns_redshifts.rate_limit_query_tns(get_data, headers, get_tns_url)

    return transient_detail

def snpy_fit(paths, plot_save=PLOTS_ROOT, model_save=SNPY_ROOT,
             model='EBV_model2', shape='st', bands=None, output=False, show_plots=False, use_saved=True):
    params = {} # mu, z, st, Tmax, EBVhost, DM
    n_targets = len(paths)
    for n in range(n_targets):
        # Set up iterable variables
        n_name = paths[n][:-9].split('/')[-1] # Splits along backslash and removes '.txt'
        n_save = model_save + n_name + '_' + model + '.snpy'
        n_path = paths[n]
        print('[ '+str(n+1)+' / '+str(n_targets)+'] Fiting data for '+n_name+'...')

        # Attempt to retrieve previous fits
        if os.path.exists(n_save) and use_saved:
            print(n_name, 'found in files! Pulling data...')
            n_s = snpy.get_sn(n_save)

        # If no saved data / used_save=False, run fit
        else:
            try:
                # Fit data -- using David configurations
                print(n_path)
                n_s = snpy.get_sn(n_path)
                n_s.choose_model(model, stype=shape)
                n_s.set_restbands() # Auto pick appropriate rest-bands
                fitargs = {'mangle':1,'calibration':0} # I don't remember what calibration is
                n_s.fit(bands=bands, dokcorr=True, k_stretch=False, reset_kcorrs=True, **fitargs)
            except:
                # If error, return None
                n_s = None
                PROBLEM_CHILDREN.append(n_name)
                print('[!!!] Unknown Error... skipping\n')

        # If fit was successful, save
        if n_s is not None:
            n_mu = n_s.get_distmod()

            # Save Model
            n_s.save(n_save)

            # Save paramaters - mu, z, st, Tmax, EBVhost, DM
            params.update({n_name : [n_mu, n_s.z, n_s.st, n_s.Tmax, n_s.EBVhost, n_s.DM]})

            # Print output
            if output:
                # Print fit summary
                n_s.summary()

                # Print paramaters
                print('Redshift:\t z = '+str(n_s.z))
                print('Distance: \t mu = '+str(round(n_mu, 4))+'\n')

            # Plotting
            n_s.plot(outfile=plot_save+n_name+'_snpyplots.png')
            if show_plots:
                plt.show()
            else:
                plt.close()

    print('Problem children:\n', '[', len(PROBLEM_CHILDREN), ']', PROBLEM_CHILDREN)
    plt.clf()
    return params

def read_DR3(loc='/content/HiloCATsSN1991bg/DR3_fits.dat'):
    data = np.genfromtxt(loc, dtype=str, skip_header=1)
    dr3 = {}
    for n in range(len(data[:, 0])):
        dr3.update({data[:, 0][n]: {'st': float(data[:, 1][n]), 'e_st': float(data[:, 2][n]), 'z': float(data[:, 3][n]),
                           'Tmax': float(data[:, 5][n]), 'e_Tmax': float(data[:, 6][n]),
                           'EBVHost': float(data[:, 25][n]), 'e_EBVHost': float(data[:, 26][n])}})
    return dr3

def dict_unpacker(path, delimiter=', '):
    with open(path, 'r') as f:
        hdr = f.readline()[:-1].split(delimiter)

    data = np.genfromtxt(path, delimiter=delimiter, dtype=str, skip_header=1)
    temp_objs = {}
    for i in range(len(data[:, 0])):
        obj = data[:, 0][i]
        temp_objs.update({obj: {}})
        for j in range(len(hdr)):
            temp_objs[obj].update({hdr[j]: data[i, j]})
    return temp_objs

def dict_packer(data_dict, save_loc, delimiter=', '):
    with open(save_loc, 'w') as f:
        for category in list(data_dict.keys()):
            f.write(category+delimiter)
        f.write('\n')
        for name in data_dict:
            f.write(name+delimiter)
            for category in list(data_dict.keys()):
                f.write(str(data_dict[name][category])+delimiter)
            f.write('\n')
    return



In [ ]:
def BurnsVDR3():
    USE_SAVED = False

    # Get Chris Burns Data
    objnames = np.genfromtxt('/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt', dtype=str, delimiter=', ')

    # Get CSP paths of Chris Burns Data
    objpaths = []
    for name in objnames:
        if os.path.exists('/content/HiloCATsSN1991bg/data/CSPdata/SN'+name+'_snpy.txt'):
            objpaths.append('/content/HiloCATsSN1991bg/data/CSPdata/SN'+name+'_snpy.txt')
        else:
            print(name, 'not found...')

    # Fitting with SNooPy
    objs = {}
    for n_path in objpaths:
        objname = n_path[39:-9]
        try:
            n_s = snpy.get_sn(n_path)
            n_s.choose_model('EBV_model2', stype='st')
            n_s.set_restbands() # Auto pick appropriate rest-bands
            fitargs = {'mangle':1,'calibration':0} # I don't remember what calibration is
            n_s.fit(bands=None, dokcorr=True, k_stretch=False, reset_kcorrs=True, **fitargs)
        except:
            print(objname+'\t| SNooPy failed to fit...')
            continue
        print(objname, '\t| mu:', n_s.get_distmod(), 'z=', n_s.z, 'st=', n_s.st, 'Tmax=', n_s.Tmax, 'EBVHost=', n_s.EBVhost)
        objs.update({objname: {'mu': n_s.get_distmod(), 'z': n_s.z, 'st': n_s.st, 'Tmax': n_s.Tmax, 'EBVHost': n_s.EBVhost}})

    # Residuals (SNooPy - DR3)
    st_res, Tmax_res, EBVHost_res = [], [], []
    st, Tmax, EBVHost = [], [], []
    dr3 = read_DR3()
    for obj in objs:
        if obj in dr3:
            st.append(objs[obj]['st'])
            Tmax.append(objs[obj]['Tmax'])
            EBVHost.append(objs[obj]['EBVHost'])

            st_res.append(objs[obj]['st'] - dr3[obj]['st'])
            Tmax_res.append(objs[obj]['Tmax'] - dr3[obj]['Tmax'])
            EBVHost_res.append(objs[obj]['EBVHost'] - dr3[obj]['EBVHost'])


    # Plot
    ## Plot reg
    reg_params = [st, Tmax, EBVHost]
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    titles = ['st', 'SNooPy Parameters \nTmax', 'EBVhost']
    for i in range(len(titles)):
        ax[i].hist(res_params[i], bins=30)
        ax[i].set_title(titles[i])
        ax[i].set_xlim(-np.max(np.abs(res_params[i])), np.max(np.abs(res_params[i])))
    plt.show()

    ## Plot residuals
    res_params = [st_res, Tmax_res, EBVHost_res]
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    titles = ['st', 'Residuals of SNooPy - Chris Burns Parameters \nTmax', 'EBVhost']
    for i in range(len(titles)):
        ax[i].hist(res_params[i], bins=30)
        ax[i].set_title(titles[i])
        ax[i].set_xlim(-np.max(np.abs(res_params[i])), np.max(np.abs(res_params[i])))
    plt.show()

    return

In [ ]:
""" MAIN """
if __name__ == '__main__':
    # Runtime tracker
    start = time.time()

    # Recovering vital directories
    recover_dir()

    BurnsVDR3()

    print('|---------------------------|\n Run-time: ', round(time.time()-start, 4), 'seconds\n|---------------------------|')